In [1]:
import time 
start_alltime = time.time()## точка отсчета времени
import cv2
import numpy as np
import os as os
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.metrics import *
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing

In [2]:
def fos(f):
    mask = np.ones(f.shape)
    f  = f.astype(np.uint8)
    mask = mask.astype(np.uint8)
    level_min = 0
    level_max = 255
    Ng = (level_max - level_min) + 1
    bins = Ng
    
    # 3) Calculate Histogram H inside ROI
    f_ravel = f.ravel() 
    mask_ravel = mask.ravel() 
    roi = f_ravel[mask_ravel.astype(bool)] 
    H = np.histogram(roi, bins=bins, range=[level_min, level_max], density=True)[0]
    
    # 4) Calculate Features
    features = np.zeros(16,np.double)  
    i = np.arange(0,bins)
    features[0] = np.dot(i,H) #Mean
    features[1] = sum(np.multiply(((i-features[0])**2),H)) #Variance
    features[2] = np.percentile(roi,50) #Median
    features[3] = np.argmax(H) #Mode
    features[4] = sum(np.multiply(((i-features[0])**3),H))/(np.sqrt(features[1])**3) #Skewness
    features[5] = sum(np.multiply(((i-features[0])**4),H))/(np.sqrt(features[1])**4) #Kurtosis
    features[6] = sum(np.multiply(H,H)) #Energy
    features[7] = -sum(np.multiply(H,np.log(H+1e-16))) #Entropy
    features[8] = min(roi) #MinimalGrayLevel
    features[9] = max(roi) #MaximalGrayLevel
    features[10] = np.sqrt(features[2]) / features[0] #CoefficientOfVariation
    features[11] = np.percentile(roi,10) # 10Percentile
    features[12] = np.percentile(roi,25) # 25Percentile
    features[13] = np.percentile(roi,75) # 75Percentile
    features[14] = np.percentile(roi,90) # 90Percentile
    features[15] = features[14] - features[11] # HistogramWidth
    
    return features

## Load and preprocessing image ##

In [3]:
def LaP_images(path):
    images = []
    filenames = os.listdir(path)
    for filename in filenames:    
        image = cv2.imread(os.path.join(path, filename),0)
        image = cv2.resize(image, dsize=(150,150))
        images.append(image)
    return np.array(images)

In [4]:
start_LaP_time = time.time()## точка отсчета времени
foil_train_dir = os.getcwd() + "\\Desert"
corduroy_train_dir = os.getcwd() + "\\DenseResidential"
cork_train_dir = os.getcwd() + "\\Forest"
catton_train_dir = os.getcwd() + "\\Meadow"
linen_train_dir = os.getcwd() + "\\River"

In [5]:
foil_train_list = LaP_images(foil_train_dir)
corduroy_train_list = LaP_images(corduroy_train_dir)
cork_train_list = LaP_images(cork_train_dir)
catton_train_list = LaP_images(catton_train_dir)
linen_train_list = LaP_images(linen_train_dir)

In [6]:
print(len(foil_train_list[1]))
print(len(foil_train_list))
print(len(corduroy_train_list))
print(len(cork_train_list))
print(len(catton_train_list))
print(len(linen_train_list))

150
217
217
217
217
217


### All image in one array ###

In [7]:
start_EX_time = time.time()## точка отсчета времени
all_images = np.vstack((foil_train_list,
                        corduroy_train_list,
                        cork_train_list,
                        catton_train_list,
                        linen_train_list
                        ))
print(all_images.shape,"Shape before extract")

(1085, 150, 150) Shape before extract


In [8]:
labels = []
for i in range(len(foil_train_list)):
    labels.append(0)
for i in range(len(corduroy_train_list)):
    labels.append(1)
for i in range(len(cork_train_list)):
    labels.append(2)
for i in range(len(catton_train_list)):
    labels.append(3)
for i in range(len(linen_train_list)):
    labels.append(4)
    
labels = np.array(labels)
print("--- %s seconds Load and preprocessing ---" % (time.time() - start_LaP_time)) ## вывод время работы программы

--- 3.886530876159668 seconds Load and preprocessing ---


## Extract First order statistic features ##

In [9]:
def extract_stat(images):
    stats = []
    for image in images: 
        stat = fos(image)
        stats.append(stat)
    
    return np.array(stats)

In [10]:
start_EX_time = time.time()## точка отсчета времени
X_all_stats = extract_stat(all_images)
print("--- %s seconds Extract Statistic Features ---" % (time.time() - start_EX_time)) ## вывод время работы программы

--- 4.050679922103882 seconds Extract Statistic Features ---


In [11]:
print(X_all_stats.shape,"Shape after extract")

(1085, 16) Shape after extract


## General dataset ##

In [12]:
col = ["Mean","Variance","Median","Mode","Skewness",
              "Kurtosis","Energy","Entropy","MinimalGrayLevel",
              "MaximalGrayLevel","CoefficientOfVariation",
              "10Percentile","25Percentile","75Percentile",
              "90Percentile","HistogramWidth"]

In [13]:
result = pd.DataFrame(data = X_all_stats,columns = col)

In [14]:
result

,Mean,Variance,Median,Mode,Skewness,Kurtosis,Energy,Entropy,MinimalGrayLevel,MaximalGrayLevel,CoefficientOfVariation,10Percentile,25Percentile,75Percentile,90Percentile,HistogramWidth
0,164.241569,41.579720,164.0,165.0,-0.466834,5.787511,0.048737,3.251443,112.0,196.0,0.077972,156.0,160.0,167.0,171.0,15.0
1,159.950674,242.952696,158.0,155.0,0.342867,3.715158,0.020368,4.135515,94.0,217.0,0.078586,142.0,150.0,168.0,180.0,38.0
2,182.640640,108.833271,182.0,181.0,-0.686278,3.679425,0.032519,3.682934,129.0,209.0,0.073865,166.0,177.0,188.0,194.0,28.0
3,147.989997,255.864160,151.0,155.0,-1.721893,6.581308,0.024553,3.985497,52.0,183.0,0.083034,127.0,142.0,158.0,163.0,36.0
4,182.608024,292.268068,181.0,180.0,-1.844239,12.965756,0.020511,4.101685,54.0,227.0,0.073675,166.0,173.0,194.0,202.0,36.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,92.015168,730.808148,87.0,107.0,0.440534,2.958040,0.012121,4.637002,7.0,218.0,0.101368,60.0,71.0,110.0,127.0,67.0
1081,113.889458,886.467522,113.0,113.0,0.092259,3.162452,0.011093,4.769729,19.0,237.0,0.093337,73.0,97.0,129.0,154.0,81.0
1082,111.447430,917.184037,116.0,128.0,-0.272535,2.100217,0.009971,4.739964,37.0,219.0,0.096640,68.0,84.0,134.0,147.0,79.0
1083,108.222299,1387.402609,95.0,88.0,1.063301,3.369861,0.011153,4.790061,36.0,247.0,0.090063,71.0,83.0,122.0,174.0,103.0


## Classification ##

In [15]:
X = result

scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

Y = labels

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size = 0.30)

In [16]:
start_SVM_time = time.time()## точка отсчета времени
clf_svm = svm.SVC()
param_grid = {'C':[150,200,250],'gamma':[0.001,0.01,0.1]}#param_grid = {'C':[5,50,100,150,200,250,300,350,400],'gamma':[0.01,0.1,0.5]}
grid_search_svm_clf=GridSearchCV(clf_svm,param_grid, refit = True, verbose=1)
grid_search_svm_clf.fit(X_train, Y_train)
print("--- %s seconds SVM+GridSearch fit ---" % (time.time() - start_SVM_time)) ## вывод время работы программы

Fitting 5 folds for each of 9 candidates, totalling 45 fits
--- 0.4704275131225586 seconds SVM+GridSearch fit ---


In [17]:
grid_search_svm_clf.best_estimator_

SVC(C=250, gamma=0.01)

In [18]:
start_PR_time = time.time()## точка отсчета времени
best_svm = grid_search_svm_clf.best_estimator_
Y_pred_svm = best_svm.predict(X_test)
print("Accuracy:",accuracy_score(Y_test, Y_pred_svm))
print(classification_report(Y_test, Y_pred_svm))
print("--- %s seconds Predict time ---" % (time.time() - start_PR_time)) ## вывод время работы программы

Accuracy: 0.8404907975460123
              precision    recall  f1-score   support

           0       0.98      0.97      0.98        64
           1       0.84      0.82      0.83        77
           2       0.77      0.88      0.82        68
           3       0.88      0.84      0.86        58
           4       0.74      0.68      0.71        59

    accuracy                           0.84       326
   macro avg       0.84      0.84      0.84       326
weighted avg       0.84      0.84      0.84       326

--- 0.010009765625 seconds Predict time ---


In [19]:
print("--- %s seconds ALL  ---" % (time.time() - start_alltime)) ## вывод время работы программы

--- 9.96605372428894 seconds ALL  ---


In [20]:
start_KNN_time = time.time()## точка отсчета времени

model_knn = KNeighborsClassifier(n_neighbors=1)
model_knn.fit(X_train, Y_train)

print('KNN train acc\t:', model_knn.score(X_train, Y_train))
print('KNN test acc\t:', model_knn.score(X_test, Y_test))

print("--- %s seconds KNN ---" % (time.time() - start_KNN_time)) ## вывод время работы программы

KNN train acc	: 1.0
KNN test acc	: 0.8159509202453987
--- 0.0990898609161377 seconds KNN ---


In [21]:
start_TEST_time = time.time()## точка отсчета времени

foil_test_dir = os.getcwd() + "\\Desert_test"
corduroy_test_dir = os.getcwd() + "\\DenseResidential_test"
cork_test_dir = os.getcwd() + "\\Forest_test"
catton_test_dir = os.getcwd() + "\\Meadow_test"
linen_test_dir = os.getcwd() + "\\River_test"


foil_test_list = LaP_images(foil_test_dir)
corduroy_test_list = LaP_images(corduroy_test_dir)
cork_test_list = LaP_images(cork_test_dir)
catton_test_list = LaP_images(catton_test_dir)
linen_test_list = LaP_images(linen_test_dir)


all_test_images = np.vstack((foil_test_list,
                        corduroy_test_list,
                        cork_test_list,
                        catton_test_list,
                        linen_test_list
                        ))

labels_test = []
for i in range(len(foil_test_list)):
    labels_test.append(0)
for i in range(len(corduroy_test_list)):
    labels_test.append(1)
for i in range(len(cork_test_list)):
    labels_test.append(2)
for i in range(len(catton_test_list)):
    labels_test.append(3)
for i in range(len(linen_test_list)):
    labels_test.append(4)

    
labels_test = np.array(labels_test)

In [22]:
X_test_stat = extract_stat(all_test_images)
result_test = pd.DataFrame(data = X_test_stat,columns = col)

In [23]:
result_test

,Mean,Variance,Median,Mode,Skewness,Kurtosis,Energy,Entropy,MinimalGrayLevel,MaximalGrayLevel,CoefficientOfVariation,10Percentile,25Percentile,75Percentile,90Percentile,HistogramWidth
0,174.069827,67.795086,173.0,173.0,-0.186613,2.674272,0.034460,3.517691,146.0,197.0,0.075561,163.0,168.0,179.0,185.0,22.0
1,142.240471,164.917117,141.0,142.0,0.060969,3.111533,0.022255,3.970161,59.0,192.0,0.083481,126.0,133.0,150.0,158.0,32.0
2,155.725681,78.980997,155.0,155.0,-0.019853,3.436099,0.032966,3.600412,107.0,195.0,0.079948,144.0,149.0,161.0,166.0,22.0
3,168.671718,269.862723,164.0,160.0,0.624620,3.151304,0.021525,4.102978,104.0,220.0,0.075924,151.0,157.0,176.0,194.0,43.0
4,153.897875,81.985884,155.0,155.0,-1.904076,8.320580,0.040976,3.452746,99.0,179.0,0.080897,143.0,150.0,159.0,162.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,71.664116,667.671754,62.0,61.0,1.680123,6.010683,0.029051,3.859704,24.0,247.0,0.109874,51.0,56.0,71.0,114.0,63.0
161,72.359900,617.909503,65.0,62.0,1.893219,8.733851,0.024939,4.069044,17.0,253.0,0.111419,52.0,58.0,74.0,113.0,61.0
162,86.339798,226.295173,86.0,95.0,0.273271,4.901271,0.020157,4.110642,32.0,205.0,0.107408,67.0,76.0,96.0,103.0,36.0
163,79.127715,210.748767,78.0,72.0,0.340978,3.204809,0.020154,4.083167,28.0,158.0,0.111614,61.0,69.0,88.0,98.0,37.0


In [24]:
start_TEPR_time = time.time()## точка отсчета времени
Xtest = result_test

scaler = preprocessing.StandardScaler().fit(Xtest)
X_sc_test = scaler.transform(Xtest)

Ytest = labels_test

In [25]:
Y_pred_test = best_svm.predict(X_sc_test)

In [26]:
print(classification_report(Ytest, Y_pred_test))

              precision    recall  f1-score   support

           0       0.89      1.00      0.94        33
           1       0.92      1.00      0.96        33
           2       1.00      0.91      0.95        33
           3       0.88      0.88      0.88        33
           4       1.00      0.88      0.94        33

    accuracy                           0.93       165
   macro avg       0.94      0.93      0.93       165
weighted avg       0.94      0.93      0.93       165



In [27]:
print("--- %s seconds Test_predict ---" % (time.time() - start_TEPR_time))
print("--- %s seconds test ---" % (time.time() - start_TEST_time))

--- 0.04804348945617676 seconds Test_predict ---
--- 1.3081884384155273 seconds test ---
